# Turning OCR lables into a table, based on bounding box coords

In [1]:
#import libraries
import json
import pandas as pd
import itertools
import collections
import statistics
import numpy as np

In [9]:
#load data and poke around a bit
f = open("HP_out/sea19631966196800natib_0008.handprint-microsoft.json")
data = json.load(f)

In [10]:
data["analyzeResult"]["readResults"][0]['lines'][4]

{'appearance': {'style': {'confidence': 0.909, 'name': 'handwriting'}},
 'boundingBox': [1122, 192, 1159, 193, 1157, 219, 1121, 218],
 'text': '154',
 'words': [{'boundingBox': [1121, 192, 1157, 193, 1157, 219, 1121, 218],
   'confidence': 0.997,
   'text': '154'}]}

In [11]:
for i in data['analyzeResult']['readResults'][0]['lines']:
    print(i['boundingBox'])
    print(i['text'])

[487, 180, 483, 215, 465, 213, 469, 178]
No
[147, 183, 418, 186, 417, 218, 146, 215]
2374 Pt. h. NigriPENNis
[515, 179, 616, 180, 616, 215, 515, 214]
21 July
[755, 187, 988, 186, 988, 212, 755, 214]
0 0 24' N 1760 30'W
[1122, 192, 1159, 193, 1157, 219, 1121, 218]
154
[1237, 189, 1288, 186, 1289, 208, 1238, 209]
mod
[1366, 177, 1479, 176, 1480, 193, 1366, 196]
LT 2X / MM
[1365, 199, 1488, 198, 1489, 215, 1365, 217]
R+ 2x1mm
[1707, 194, 1749, 196, 1749, 215, 1706, 214]
BS
[1818, 193, 1882, 194, 1882, 215, 1818, 214]
SKIN
[1916, 189, 2041, 192, 2040, 218, 1916, 214]
Bulmer 937
[135, 227, 207, 227, 207, 251, 135, 250]
2375
[242, 223, 457, 227, 457, 257, 242, 254]
PattiNis pacificus
[762, 227, 997, 228, 997, 252, 762, 251]
00 38' N 1760 35' W
[1106, 228, 1164, 230, 1164, 254, 1106, 253]
457
[1248, 231, 1304, 231, 1305, 253, 1248, 254]
Light
[1606, 222, 1629, 222, 1630, 237, 1607, 237]
M
[1380, 237, 1552, 236, 1552, 254, 1380, 256]
725 /75 4 X4mm
[1715, 237, 1744, 236, 1744, 254, 1715, 255]


In [12]:
len(data["analyzeResult"]["readResults"][0]['lines'][4]['words'])

1

In [13]:
word_count = []
for l in data["analyzeResult"]["readResults"][0]['lines']:
    word_count.append(len(l['words']))

In [14]:
max(word_count)

7

## General strategy

Generally, the columns are spaced further apart from each other than the rows, and there's fewer columns than rows. So we're going to build the table by first creating a set of columns based on where the x coords fall in space. We're assuming that all of the boxes will overlap along the x axis at least a little bit. It looks like the numbering column (column 1) largely counts all the rows and is reliably identified by the HTR algorithm, therefore we can base all rows on the y coords from the boxes in the first column.

1. Create dataframes for each "column" with the bbox number, the text, and the average of the y coords. 
2. Create a new table to populate with just the text data. Add column 1 to determine number of rows.
3. Check if the y_avg for a given box is between the y_top and y_bot of the first column values, if so add it to the rogt index (row number) in a list 
4. Add the list to the table as a new column
5. ...profit?

**Notes**
- Works well for some images, but breaks down when detected "lines" cross intellectual columns. E.g. "HP_out/sea19631966196800natib_0008.handprint-microsoft.json" line 1 contains a value in column 1 and column 2. Could move down to sorting boxes at "word" level.
- May need to refactor to assign row values directly to dataframe, since the list comprehension is just adding another convoluted step to create lists of lists with placeholders. 
- Create functions to reduce repetativeness
    - Create single column or create all columns? (get first column and keep computing columns based on previous one as long as there are still boxes that are further on the xaxis than the column just created. 
    - Create row? 
    - One giant function? table_from_boxes(filename)? Not as wild as it sounds since all the placements are computed based on the locations of the first column. 
    

### Column 1

In [15]:
#create empty lists
zero = []
six = []

#add leftmost x-axis bbox coords (index 0 & 6) to list
for i in data['analyzeResult']['readResults'][0]['lines']:
    zero.append(i['boundingBox'][0])
    six.append(i['boundingBox'][6])

#create dataframe with coords (index = bbox #) 
xleft = pd.DataFrame(
    {'zero': zero,
     'six' : six
    })

#find minimum (leftmost) of left coords from bbox
min_zero = xleft.min()[0]
min_six = xleft.min()[1]
xmin_left_1 = min(min_zero, min_six)

#find row/index of minimum x value for left bbox coords
m = xleft.idxmin()
m = m[0]

#find maximum cooresponding x value for right side of minimum bbox)
two = data['analyzeResult']['readResults'][0]['lines'][m]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][m]['boundingBox'][4]
xmax_right_1 = max(two, four)

In [16]:
data['analyzeResult']['readResults'][0]['lines'][m]

{'appearance': {'style': {'confidence': 0.676, 'name': 'handwriting'}},
 'boundingBox': [135, 227, 207, 227, 207, 251, 135, 250],
 'text': '2375',
 'words': [{'boundingBox': [137, 227, 203, 228, 202, 252, 136, 251],
   'confidence': 0.993,
   'text': '2375'}]}

In [17]:
#find all bboxes with a left x value between the minimum left x value and the maximum right x value - this should give you all the bboxes that overlap with the farthest left bbox (aka column zero)
number = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if xmin_left_1 <= value['boundingBox'][0] < xmax_right_1:
        number[index] = value['text']
        continue

#turn dict into data frame so you can add column for y_avg
number_df = pd.DataFrame(list(number.items()), columns = ['box','line'])
number_df

,box,line
0,1,2374 Pt. h. NigriPENNis
1,11,2375
2,20,76
3,29,77
4,35,78
5,40,79
6,47,80
7,58,81
8,59,82
9,69,83


### Column 2

In [18]:
#find all xleft values larger than the largest xleft in col 1
xleft_2 = xleft[xleft >= (xmax_right_1 + 1)]

#find index of smallest x_left larger than xmax_right_1 + 1
n = xleft_2.idxmin()
n = n[0]

#find values of bbox coords 2 & 4 of smallest x_left box
two = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][4]
n_max = max(two, four)

#find values of bbox coords 0 & 6 that are smaller than n_max
xmin_left_2 = xleft_2[xleft_2 <= (n_max+50)] #add 100 to account for a small box for xmax
xmin_left_2 = xmin_left_2.dropna()
xmin_left_2

#Find coords 2 & 4 of all bboxes in xmin_left_2
xmax_right_2 = []

for i,j in xmin_left_2.iterrows():
    xmax_right_2.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][2])
    xmax_right_2.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][4])

#find max right x coord
xmax_right_2 = max(xmax_right_2)

In [19]:
#find all bboxes with a 0 coord between xmax_right_1 + 1 and max possible right coord to find all potential overlappping bboxes
name = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if (xmax_right_1 + 1) < data['analyzeResult']['readResults'][0]['lines'][index]['boundingBox'][0] < xmax_right_2:
        name[index] = value['text']
    else:
        continue

name_df = pd.DataFrame(list(name.items()), columns = ['box','line'])
name_df

,box,line
0,0,No
1,12,PattiNis pacificus
2,21,StarNA fuscAtA
3,30,Fragata Ariil
4,41,StirNA fusente
5,42,-
6,50,ProcolostarNA crowdER
7,51,-
8,60,phasthow laptrus
9,70,-


### Column 3

In [20]:
#find all xleft values larger than the largest xleft in col 2
xleft_3 = xleft_2[xleft_2 > (xmax_right_2 + 1)]

#find index of smallest x_left larger than xmax_right_2 + 1
n = xleft_3.idxmin()
n = n[0]

#find values of bbox coords 2 & 4 of n
two = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][4]
n_max = max(two, four)

#find values of bbox coords 0 & 6 that are smaller than n_max
xmin_left_3 = xleft_3[xleft_3 < n_max]
xmin_left_3 = xmin_left_3.dropna()
xmin_left_3

#Find coords 2 & 4 of all bboxes in xmin_left_3
xmax_right_3 = []

for i,j in xmin_left_3.iterrows():
    xmax_right_3.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][2])
    xmax_right_3.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][4])

#find max right x coord
xmax_right_3 = max(xmax_right_3)

In [21]:
sex = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if (xmax_right_2 + 1) < data['analyzeResult']['readResults'][0]['lines'][index]['boundingBox'][0] < xmax_right_3:
        sex[index] = value['text']
    else:
        continue

sex_df = pd.DataFrame(list(sex.items()), columns = ['box','line'])
sex_df.head(10)

,box,line
0,2,21 July
1,22,22 dny
2,43,23 Jily
3,56,=
4,61,27 duly
5,71,3 Aug
6,84,=
7,89,4 5Aug.
8,112,8 Aug
9,125,10 Aug.


### Column 4

In [22]:
#find all xleft values larger than the largest xleft in col 2
xleft_4 = xleft_3[xleft_3 > (xmax_right_3 + 1)]

#find index of smallest x_left larger than xmax_right_2 + 1
n = xleft_4.idxmin()
n = n[0]

#find values of bbox coords 2 & 4 of n
two = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][4]
n_max = max(two, four)

#find values of bbox coords 0 & 6 that are smaller than n_max
xmin_left_4 = xleft_4[xleft_4 < n_max]
xmin_left_4 = xmin_left_4.dropna()
xmin_left_4

#Find coords 2 & 4 of all bboxes in xmin_left_3
xmax_right_4 = []

for i,j in xmin_left_4.iterrows():
    xmax_right_4.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][2])
    xmax_right_4.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][4])

#find max right x coord
xmax_right_4 = max(xmax_right_4)

In [23]:
date = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if (xmax_right_3 + 1) < data['analyzeResult']['readResults'][0]['lines'][index]['boundingBox'][0] < xmax_right_4:
        date[index] = value['text']
    else:
        continue

date_df = pd.DataFrame(list(date.items()), columns = ['box','line'])
date_df.head()

,box,line
0,3,0 0 24' N 1760 30'W
1,13,00 38' N 1760 35' W
2,23,60 27's 1750 57W
3,31,60 35's 1750 52'W
4,36,00 35' 5 1750 5l'W


### Column 5 

In [24]:
#find all xleft values larger than the largest xleft in col 2
xleft_5 = xleft_4[xleft_4 > (xmax_right_4 + 1)]

#find index of smallest x_left larger than xmax_right_2 + 1
n = xleft_5.idxmin()
n = n[0]

#find values of bbox coords 2 & 4 of n
two = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][4]
n_max = max(two, four)

#find values of bbox coords 0 & 6 that are smaller than n_max
xmin_left_5 = xleft_5[xleft_5 < n_max]
xmin_left_5 = xmin_left_5.dropna()
xmin_left_5

#Find coords 2 & 4 of all bboxes in xmin_left_3
xmax_right_5 = []

for i,j in xmin_left_5.iterrows():
    xmax_right_5.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][2])
    xmax_right_5.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][4])

#find max right x coord
xmax_right_5 = max(xmax_right_5)

In [25]:
geo = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if (xmax_right_4 + 1) < data['analyzeResult']['readResults'][0]['lines'][index]['boundingBox'][0] < xmax_right_5:
        geo[index] = value['text']
    else:
        continue

geo_df = pd.DataFrame(list(geo.items()), columns = ['box','line'])
geo_df.head()

,box,line
0,4,154
1,14,457
2,24,192
3,53,49
4,63,325


### Column 6

In [26]:
#find all xleft values larger than the largest xleft in col 2
xleft_6 = xleft_5[xleft_5 > (xmax_right_5 + 1)]

#find index of smallest x_left larger than xmax_right_2 + 1
n = xleft_6.idxmin()
n = n[0]

#find values of bbox coords 2 & 4 of n
two = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][4]
n_max = max(two, four)

#find values of bbox coords 0 & 6 that are smaller than n_max
xmin_left_6 = xleft_6[xleft_6 < n_max]
xmin_left_6 = xmin_left_6.dropna()
xmin_left_6

#Find coords 2 & 4 of all bboxes in xmin_left_3
xmax_right_6 = []

for i,j in xmin_left_6.iterrows():
    xmax_right_6.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][2])
    xmax_right_6.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][4])

#find max right x coord
xmax_right_6 = max(xmax_right_6)

In [27]:
weight = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if (xmax_right_5 + 1) < data['analyzeResult']['readResults'][0]['lines'][index]['boundingBox'][0] < xmax_right_6:
        weight[index] = value['text']
    else:
        continue

weight_df = pd.DataFrame(list(weight.items()), columns = ['box','line'])
weight_df.head()

,box,line
0,5,mod
1,6,LT 2X / MM
2,7,R+ 2x1mm
3,15,Light
4,17,725 /75 4 X4mm


### Column 7

In [28]:
#find all xleft values larger than the largest xleft in col 2
xleft_7 = xleft_6[xleft_6 > (xmax_right_6 + 1)]

#find index of smallest x_left larger than xmax_right_2 + 1
n = xleft_7.idxmin()
n = n[0]

#find values of bbox coords 2 & 4 of n
two = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][4]
n_max = max(two, four)

#find values of bbox coords 0 & 6 that are smaller than n_max
xmin_left_7 = xleft_7[xleft_7 < n_max]
xmin_left_7 = xmin_left_7.dropna()
xmin_left_7

#Find coords 2 & 4 of all bboxes in xmin_left_3
xmax_right_7 = []

for i,j in xmin_left_7.iterrows():
    xmax_right_7.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][2])
    xmax_right_7.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][4])

#find max right x coord
xmax_right_7 = max(xmax_right_7)

In [29]:
fat_con = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if (xmax_right_6 + 1) < data['analyzeResult']['readResults'][0]['lines'][index]['boundingBox'][0] < xmax_right_7:
        fat_con[index] = value['text']
    else:
        continue

fat_con_df = pd.DataFrame(list(fat_con.items()), columns = ['box','line'])
fat_con_df.head()

,box,line
0,8,BS
1,9,SKIN
2,10,Bulmer 937
3,16,M
4,18,3 S


### Column 8

In [30]:
#find all xleft values larger than the largest xleft in col 2
xleft_8 = xleft_7[xleft_7 > (xmax_right_7 + 1)]

#find index of smallest x_left larger than xmax_right_2 + 1
n = xleft_8.idxmin()
n = n[0]

#find values of bbox coords 2 & 4 of n
two = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][4]
n_max = max(two, four)

#find values of bbox coords 0 & 6 that are smaller than n_max
xmin_left_8 = xleft_8[xleft_8 < n_max]
xmin_left_8 = xmin_left_8.dropna()
xmin_left_8

#Find coords 2 & 4 of all bboxes in xmin_left_3
xmax_right_8 = []

for i,j in xmin_left_8.iterrows():
    xmax_right_8.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][2])
    xmax_right_8.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][4])

#find max right x coord
xmax_right_8 = max(xmax_right_8)

TypeError: list indices must be integers or slices, not numpy.float64

In [265]:
reproduction = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if (xmax_right_7 + 1) < data['analyzeResult']['readResults'][0]['lines'][index]['boundingBox'][0] < xmax_right_8:
        reproduction[index] = value['text']
    else:
        continue

reproduction_df = pd.DataFrame(list(reproduction.items()), columns = ['box','line'])
reproduction_df.head()

,box,line
0,33,BP & RE finthand
1,53,BP Present
2,54,TIL9x 11 mm
3,61,RT 18X / 3MM
4,70,L. OUum 2x 2mm


### Column 9

In [266]:
#find all xleft values larger than the largest xleft in col 2
xleft_9 = xleft_8[xleft_8 > (xmax_right_8 + 1)]

#find index of smallest x_left larger than xmax_right_2 + 1
n = xleft_9.idxmin()
n = n[0]

#find values of bbox coords 2 & 4 of n
two = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][4]
n_max = max(two, four)

#find values of bbox coords 0 & 6 that are smaller than n_max
xmin_left_9 = xleft_9[xleft_9 < n_max]
xmin_left_9 = xmin_left_9.dropna()
xmin_left_9

#Find coords 2 & 4 of all bboxes in xmin_left_3
xmax_right_9 = []

for i,j in xmin_left_9.iterrows():
    xmax_right_9.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][2])
    xmax_right_9.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][4])

#find max right x coord
xmax_right_9 = max(xmax_right_9)

In [267]:
remarks = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if (xmax_right_8 + 1) < data['analyzeResult']['readResults'][0]['lines'][index]['boundingBox'][0] < xmax_right_9:
        remarks[index] = value['text']
    else:
        continue

remarks_df = pd.DataFrame(list(remarks.items()), columns = ['box','line'])
remarks_df.head()

,box,line
0,34,Fus # 843-39034
1,39,18luz print
2,40,S
3,55,M .
4,56,C.N .


### Column 10

In [268]:
#find all xleft values larger than the largest xleft in col 2
xleft_10 = xleft_9[xleft_9 > (xmax_right_9 + 1)]

#find index of smallest x_left larger than xmax_right_2 + 1
n = xleft_10.idxmin()
n = n[0]

#find values of bbox coords 2 & 4 of n
two = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][4]
n_max = max(two, four)

#find values of bbox coords 0 & 6 that are smaller than n_max
xmin_left_10 = xleft_10[xleft_10 < n_max]
xmin_left_10 = xmin_left_10.dropna()
xmin_left_10

#Find coords 2 & 4 of all bboxes in xmin_left_3
xmax_right_10 = []

for i,j in xmin_left_10.iterrows():
    xmax_right_10.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][2])
    xmax_right_10.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][4])

#find max right x coord
xmax_right_10 = max(xmax_right_10)

In [269]:
col_10 = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if (xmax_right_9 + 1) < data['analyzeResult']['readResults'][0]['lines'][index]['boundingBox'][0] < xmax_right_10:
        col_10[index] = value['text']
    else:
        continue

col_10_df = pd.DataFrame(list(col_10.items()), columns = ['box','line'])
col_10_df.head()

,box,line
0,6,Alco
1,11,11
2,41,SKIN
3,47,Alco
4,63,Skin


### Column 11

In [270]:
#find all xleft values larger than the largest xleft in col 2
xleft_11 = xleft_10[xleft_10 > (xmax_right_10 + 1)]

#find index of smallest x_left larger than xmax_right_2 + 1
n = xleft_11.idxmin()
n = n[0]

#find values of bbox coords 2 & 4 of n
two = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][2]
four = data['analyzeResult']['readResults'][0]['lines'][n]['boundingBox'][4]
n_max = max(two, four)

#find values of bbox coords 0 & 6 that are smaller than n_max
xmin_left_11 = xleft_11[xleft_11 < n_max]
xmin_left_11 = xmin_left_11.dropna()
xmin_left_11

#Find coords 2 & 4 of all bboxes in xmin_left_3
xmax_right_11 = []

for i,j in xmin_left_11.iterrows():
    xmax_right_11.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][2])
    xmax_right_11.append(data['analyzeResult']['readResults'][0]['lines'][i]['boundingBox'][4])

#find max right x coord
xmax_right_11 = max(xmax_right_11)

In [271]:
col_11 = {}

for index, value in enumerate(data['analyzeResult']['readResults'][0]['lines']):
    if (xmax_right_10 + 1) < data['analyzeResult']['readResults'][0]['lines'][index]['boundingBox'][0] < xmax_right_11:
        col_11[index] = value['text']
    else:
        continue

col_11_df = pd.DataFrame(list(col_11.items()), columns = ['box','line'])
col_11_df.head()

,box,line
0,7,Haber 872
1,12,87/ 8
2,13,=
3,19,376
4,24,41


## Create table to populate with column data

In [300]:
table = pd.DataFrame(columns = [
    'number',
    'name',
    'sex',
    'date',
    'geo',
    'weight',
    'fat_con',
    'reproduction',
    'remarks',
    'col_10',
    'col_11'
])

table['number'] = number_df['line']

In [301]:
table.head()

,number,name,sex,date,geo,weight,fat_con,reproduction,remarks,col_10,col_11
0,23 16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [302]:
#Add y_top, y_bottom, & y_avg to number_df
y_top = []
y_bot = []
y_avg = []

for box in number_df['box']:
    tops = []
    tops.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    tops.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    
    bottoms = []
    bottoms.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    bottoms.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])
    
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])
    
    y_top.append(statistics.mean(tops))
    y_bot.append(statistics.mean(bottoms))
    y_avg.append(statistics.mean(y_list))
    
number_df["y_top"] = y_top
number_df["y_bot"] = y_bot
number_df["y_avg"] = y_avg

In [303]:
number_df

,box,line,y_top,y_bot,y_avg
0,0,23 16,252.0,276.5,264.25
1,14,17,305.5,327.5,316.50
2,15,18,353.0,377.0,365.00
3,20,19,408.0,437.5,422.75
4,26,20,457.5,487.5,472.50
5,36,21,507.5,534.5,521.00
6,44,22,559.5,586.5,573.00
7,49,23,612.0,637.0,624.50
8,57,24,658.5,690.5,674.50
9,67,25,714.0,746.0,730.00


## Rows! 

In [304]:
#add y_avg to names_df
y_avg = []

for box in name_df['box']:
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])

    y_avg.append(statistics.mean(y_list))
    
name_df["y_avg"] = y_avg

In [305]:
names_full = ["place_holder"] * len(table)
for i, j in number_df.iterrows():
    for k, l in name_df.iterrows():
        if j['y_top'] < l['y_avg'] < j['y_bot']:
            print("Row", i, "is", l['line'])
            if len(names_full[i]) < 12:
                names_full[i].append(l['line'])
            else:
                names_full[i]=l['line']
            continue

Row 0 is SterNA fuscAtA
Row 8 is FregatA Ariel
Row 10 is Pt. h. NigripINNis
Row 11 is StarNA fuscAtA
Row 14 is PhAsthow rubricAndA
Row 15 is Pt. L. NigripsUNIT
Row 16 is StarNA fuSCATA
Row 19 is 11
Row 24 is 11


In [306]:
#Add names to table
table['name'] = names_full

In [307]:
#Sex

#Add y_avg
y_avg = []
y_left_avg = []
y_right_avg = []

for box in sex_df['box']:
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])

    y_avg.append(statistics.mean(y_list))

sex_df["y_avg"] = y_avg

#add names to table based on y_avg location
sex_full = ["place_holder"] * len(table)
for i, j in number_df.iterrows():
    for k, l in sex_df.iterrows():
        if j['y_top'] < l['y_avg'] < j['y_bot']:
            print("Row", i, "is", l['line'])
            if len(sex_full[i]) < 12:
                sex_full[i].append(l['line'])
            else:
                sex_full[i]=l['line']
            continue

Row 0 is -
Row 1 is -
Row 2 is -
Row 3 is -
Row 4 is -
Row 5 is No
Row 6 is -
Row 8 is 87
Row 9 is 0+
Row 10 is No
Row 11 is -
Row 12 is No
Row 16 is -
Row 18 is -
Row 19 is 0+
Row 23 is -
Row 24 is -
Row 26 is -


In [308]:
#Add sex to table
table['sex'] = sex_full
table

,number,name,sex,date,geo,weight,fat_con,reproduction,remarks,col_10,col_11
0,23 16,SterNA fuscAtA,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17,place_holder,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18,place_holder,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19,place_holder,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20,place_holder,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,21,place_holder,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,22,place_holder,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,23,place_holder,place_holder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,24,FregatA Ariel,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,25,place_holder,0+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [309]:
#Date

#Add y_avg
y_avg = []
y_left_avg = []
y_right_avg = []

for box in date_df['box']:
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])

    y_avg.append(statistics.mean(y_list))

date_df["y_avg"] = y_avg

#add names to table based on y_avg location
date_full = ["place_holder"] * len(table)

for i, j in number_df.iterrows():
    for k, l in date_df.iterrows():
        if j['y_top'] < l['y_avg'] < j['y_bot']:
            print("Row", i, "is", l['line'])
            if len(date_full[i]) == 12:
                date_full[i]=[l['line']]
            else:
                date_full[i].append(l['line'])
            continue

Row 0 is 17 duly
Row 2 is 11
Row 5 is =
Row 8 is =
Row 10 is =
Row 11 is 18 July
Row 15 is 1'
Row 16 is 19 July
Row 20 is 20 July
Row 21 is =
Row 23 is =
Row 24 is 21 July
Row 25 is 1!
Row 28 is 11


In [310]:
#add date to table
table['date'] = date_full

In [311]:
table

,number,name,sex,date,geo,weight,fat_con,reproduction,remarks,col_10,col_11
0,23 16,SterNA fuscAtA,-,[17 duly],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,17,place_holder,-,place_holder,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,18,place_holder,-,[11],NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,19,place_holder,-,place_holder,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20,place_holder,-,place_holder,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,21,place_holder,No,[=],NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,22,place_holder,-,place_holder,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,23,place_holder,place_holder,place_holder,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,24,FregatA Ariel,87,[=],NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,25,place_holder,0+,place_holder,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [312]:
#Geo

#Add y_avg
y_avg = []
y_left_avg = []
y_right_avg = []

for box in geo_df['box']:
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])

    y_avg.append(statistics.mean(y_list))

geo_df["y_avg"] = y_avg

#add names to table based on y_avg location
geo_full = ["place_holder"] * len(table)

for i, j in number_df.iterrows():
    for k, l in geo_df.iterrows():
        if j['y_top'] < l['y_avg'] < j['y_bot']:
            print("Row", i, "is", l['line'])
            if len(geo_full[i]) == 12:
                geo_full[i]=[l['line']]
            else:
                geo_full[i].append(l['line'])
            continue

Row 0 is 00 50'N 176 20W
Row 1 is =
Row 3 is 0048'N 176 33/0
Row 4 is 0 0 42'N 176" 16' W
Row 5 is OOHI'N 176º15'0
Row 7 is @ 30 N 1750 38' W
Row 8 is 0'27 N 1750 29/W
Row 9 is 00 27'N / 750 29/W
Row 10 is 0 0 08N 175 0 54W
Row 11 is 0º HON 1760 5210
Row 12 is 0º 38'₪ 1760 54'w
Row 13 is 035' N 1770 01 W
Row 14 is 10 14'N 1750 56'W
Row 15 is 00 03' N 1780 l'W
Row 16 is 0 0 29'N 176 0 48W
Row 17 is 00 30'N 176º 470
Row 18 is 11
Row 19 is 0º 34'₪ 176º 44'w
Row 20 is 6º OS'N 1720 41'W
Row 21 is 00 18'N 1760 41 W
Row 22 is 20 ~ 1760 41w
Row 23 is 26' N 1760 4 1'W
Row 24 is 30'N 1760 31'W
Row 25 is 28N 172 31'00
Row 26 is 0 28'N 17 6 º 31'W
Row 28 is 11


In [313]:
table['geo'] = geo_full

In [314]:
#Weight

#Add y_avg
y_avg = []
y_left_avg = []
y_right_avg = []

for box in weight_df['box']:
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])

    y_avg.append(statistics.mean(y_list))

weight_df["y_avg"] = y_avg

#add names to table based on y_avg location
weight_full = ["place_holder"] * len(table)

for i, j in number_df.iterrows():
    for k, l in weight_df.iterrows():
        if j['y_top'] < l['y_avg'] < j['y_bot']:
            print("Row", i, "is", l['line'])
            if len(weight_full[i]) == 12:
                weight_full[i]=[l['line']]
            else:
                weight_full[i].append(l['line'])
            continue

Row 0 is 159.0
Row 1 is 162.6
Row 2 is 11.3
Row 3 is 147.9
Row 4 is 166.7
Row 5 is 187
Row 6 is 203
Row 7 is 157.4
Row 10 is 871
Row 11 is 34
Row 12 is 160
Row 13 is 147
Row 14 is 920
Row 15 is 152
Row 16 is 197
Row 17 is 193
Row 18 is 184
Row 19 is 198
Row 20 is 187
Row 21 is 19 2
Row 22 is 168
Row 23 is 216
Row 24 is 79
Row 25 is 7/
Row 26 is 169
Row 27 is 197
Row 28 is 173


In [315]:
table['weight'] = weight_full

In [316]:
#fat_con

#Add y_avg
y_avg = []
y_left_avg = []
y_right_avg = []

for box in fat_con_df['box']:
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])

    y_avg.append(statistics.mean(y_list))

fat_con_df["y_avg"] = y_avg

#add names to table based on y_avg location
fat_con_full = ["place_holder"] * len(table)

for i, j in number_df.iterrows():
    for k, l in fat_con_df.iterrows():
        if j['y_top'] < l['y_avg'] < j['y_bot']:
            print("Row", i, "is", l['line'])
            if len(fat_con_full[i]) == 12:
                fat_con_full[i]=[l['line']]
            else:
                fat_con_full[i].append(l['line'])
            continue

Row 9 is Mod
Row 10 is heavy
Row 12 is Light
Row 14 is LEAvy
Row 15 is heavy
Row 19 is hlavy
Row 24 is HEAVY
Row 27 is Light
Row 28 is Light


In [317]:
table['fat_con'] = fat_con_full

In [319]:
#Reproduction

#Add y_avg
y_avg = []
y_left_avg = []
y_right_avg = []

for box in reproduction_df['box']:
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])

    y_avg.append(statistics.mean(y_list))

reproduction_df["y_avg"] = y_avg

#add names to table based on y_avg location
reproduction_full = ["place_holder"] * len(table)

for i, j in number_df.iterrows():
    for k, l in reproduction_df.iterrows():
        if j['y_top'] < l['y_avg'] < j['y_bot']:
            print("Row", i, "is", l['line'])
            if len(reproduction_full[i]) == 12:
                reproduction_full[i]=[l['line']]
            else:
                reproduction_full[i].append(l['line'])
            continue

Row 8 is TIL9x 11 mm
Row 8 is RT 18X / 3MM
Row 9 is L. OUum 2x 2mm
Row 10 is L. 15>+15 1X 2mm
Row 12 is +$ *** 5 4×3mm
Row 14 is L. ouum4min
Row 15 is 1.+25+15 1× 1 mm
Row 19 is Qvary 10x5mm
Row 24 is gowAds destroyed
Row 27 is 725 /55 5×4 0mm
Row 28 is BP complete


In [320]:
table['reproduction'] = reproduction_full

In [321]:
#Remarks

#Add y_avg
y_avg = []
y_left_avg = []
y_right_avg = []

for box in remarks_df['box']:
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])

    y_avg.append(statistics.mean(y_list))

remarks_df["y_avg"] = y_avg

#add names to table based on y_avg location
remarks_full = ["place_holder"] * len(table)

for i, j in number_df.iterrows():
    for k, l in remarks_df.iterrows():
        if j['y_top'] < l['y_avg'] < j['y_bot']:
            print("Row", i, "is", l['line'])
            if len(remarks_full[i]) == 12:
                remarks_full[i]=[l['line']]
            else:
                remarks_full[i].append(l['line'])
            continue

Row 5 is Fus # 843-39034
Row 5 is 18luz print
Row 5 is S
Row 8 is 5
Row 9 is 5
Row 10 is BS
Row 12 is 5
Row 14 is S
Row 15 is 4-014
Row 18 is CN
Row 19 is 5
Row 24 is 3 5
Row 28 is M


In [322]:
table['remarks'] = remarks_full

In [323]:
#col_10

#Add y_avg
y_avg = []
y_left_avg = []
y_right_avg = []

for box in col_10_df['box']:
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])

    y_avg.append(statistics.mean(y_list))

col_10_df["y_avg"] = y_avg

#add names to table based on y_avg location
col_10_full = ["place_holder"] * len(table)

for i, j in number_df.iterrows():
    for k, l in col_10_df.iterrows():
        if j['y_top'] < l['y_avg'] < j['y_bot']:
            print("Row", i, "is", l['line'])
            if len(col_10_full[i]) == 12:
                col_10_full[i]=[l['line']]
            else:
                col_10_full[i].append(l['line'])
            continue

Row 0 is Alco
Row 1 is 11
Row 5 is SKIN
Row 6 is Alco
Row 8 is Skin
Row 11 is Alco
Row 12 is Skal
Row 13 is Alco
Row 15 is SKIN
Row 16 is Alco
Row 17 is 1.1
Row 20 is Alco
Row 23 is =
Row 24 is Skal
Row 25 is Alco
Row 26 is 1/
Row 27 is Skal


In [324]:
table['col_10'] = col_10_full

In [325]:
#col_11

#Add y_avg
y_avg = []
y_left_avg = []
y_right_avg = []

for box in col_11_df['box']:
    y_list = []
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][1])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][3])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][5])
    y_list.append(data['analyzeResult']['readResults'][0]['lines'][box]['boundingBox'][7])

    y_avg.append(statistics.mean(y_list))

col_11_df["y_avg"] = y_avg

#add names to table based on y_avg location
col_11_full = ["place_holder"] * len(table)

for i, j in number_df.iterrows():
    for k, l in col_11_df.iterrows():
        if j['y_top'] < l['y_avg'] < j['y_bot']:
            print("Row", i, "is", l['line'])
            if len(col_11_full[i]) == 12:
                col_11_full[i]=[l['line']]
            else:
                col_11_full[i].append(l['line'])
            continue

Row 0 is Haber 872
Row 1 is 87/ 8
Row 1 is =
Row 3 is 41
Row 3 is 869.
Row 4 is 三
Row 6 is 867
Row 7 is 866
Row 8 is Chandler 11
Row 9 is Bulmer 939
Row 10 is BAKcomb 107
Row 11 is Huber 897
Row 12 is Ho€€ 56
Row 13 is Huber 878
Row 14 is Hott 4
Row 15 is Balcond 108
Row 16 is Huber 899
Row 17 is 1 1 900
Row 18 is 901
Row 19 is Hoff 57
Row 20 is Huber 902
Row 21 is 903
Row 22 is 904
Row 23 is 905
Row 24 is Hote 58
Row 25 is Huber 906
Row 26 is 907
Row 27 is Hoff 59:
Row 28 is 11 605


In [326]:
table['col_11'] = col_11_full

In [327]:
table

,number,name,sex,date,geo,weight,fat_con,reproduction,remarks,col_10,col_11
0,23 16,SterNA fuscAtA,-,[17 duly],[00 50'N 176 20W],[159.0],place_holder,place_holder,place_holder,[Alco],[Haber 872]
1,17,place_holder,-,place_holder,[=],[162.6],place_holder,place_holder,place_holder,[11],"[87/ 8, =]"
2,18,place_holder,-,[11],place_holder,[11.3],place_holder,place_holder,place_holder,place_holder,place_holder
3,19,place_holder,-,place_holder,[0048'N 176 33/0],[147.9],place_holder,place_holder,place_holder,place_holder,"[41, 869.]"
4,20,place_holder,-,place_holder,"[0 0 42'N 176"" 16' W]",[166.7],place_holder,place_holder,place_holder,place_holder,[三]
5,21,place_holder,No,[=],[OOHI'N 176º15'0],[187],place_holder,place_holder,"[Fus # 843-39034, 18luz print, S]",[SKIN],place_holder
6,22,place_holder,-,place_holder,place_holder,[203],place_holder,place_holder,place_holder,[Alco],[867]
7,23,place_holder,place_holder,place_holder,[@ 30 N 1750 38' W],[157.4],place_holder,place_holder,place_holder,place_holder,[866]
8,24,FregatA Ariel,87,[=],[0'27 N 1750 29/W],place_holder,place_holder,"[TIL9x 11 mm, RT 18X / 3MM]",[5],[Skin],[Chandler 11]
9,25,place_holder,0+,place_holder,[00 27'N / 750 29/W],place_holder,[Mod],[L. OUum 2x 2mm],[5],place_holder,[Bulmer 939]


In [329]:
#remove "place_holder"
table = table.replace("place_holder", np.nan)
table

,number,name,sex,date,geo,weight,fat_con,reproduction,remarks,col_10,col_11
0,23 16,SterNA fuscAtA,-,[17 duly],[00 50'N 176 20W],[159.0],NaN,NaN,NaN,[Alco],[Haber 872]
1,17,NaN,-,NaN,[=],[162.6],NaN,NaN,NaN,[11],"[87/ 8, =]"
2,18,NaN,-,[11],NaN,[11.3],NaN,NaN,NaN,NaN,NaN
3,19,NaN,-,NaN,[0048'N 176 33/0],[147.9],NaN,NaN,NaN,NaN,"[41, 869.]"
4,20,NaN,-,NaN,"[0 0 42'N 176"" 16' W]",[166.7],NaN,NaN,NaN,NaN,[三]
5,21,NaN,No,[=],[OOHI'N 176º15'0],[187],NaN,NaN,"[Fus # 843-39034, 18luz print, S]",[SKIN],NaN
6,22,NaN,-,NaN,NaN,[203],NaN,NaN,NaN,[Alco],[867]
7,23,NaN,NaN,NaN,[@ 30 N 1750 38' W],[157.4],NaN,NaN,NaN,NaN,[866]
8,24,FregatA Ariel,87,[=],[0'27 N 1750 29/W],NaN,NaN,"[TIL9x 11 mm, RT 18X / 3MM]",[5],[Skin],[Chandler 11]
9,25,NaN,0+,NaN,[00 27'N / 750 29/W],NaN,[Mod],[L. OUum 2x 2mm],[5],NaN,[Bulmer 939]


In [330]:
#save to test dir
table.to_csv("sea19631966196800natib_0006.csv", index = False)